In [1]:
from collections import defaultdict
from pathlib import Path

from IPython.display import (
    display,
    Markdown,
)
import pandas as pd
import seaborn as sns
import os

from howso.nominal_substitution.substitution_engine import NominalSubstitutionEngine
from howso.synthesizer import Synthesizer
from howso.utilities import infer_feature_attributes
from howso.validator import Validator
from howso.validator.results import ValidatorResult


os.environ["LANG"] = "en_US.UTF-8"
os.environ["LC_ALL"] = "en_US.UTF-8"

# Set this to True if the dataset is deemed to already have been anonymized.
ALREADY_ANONYMIZED: bool = True

In [2]:
col_type_mapping = {
    'nih_age': 'Int64', 'nih_alcohol_frequency': 'Int64', 'alcohol_drinksperday': 'Int64',
    'nih_sex': 'Int64', 'nih_asthma': 'Int64', 'nih_alc_sub_abuse': 'Int64', 'nih_autoimm': 'Int64',
    'nih_cancer_past_yr': 'Int64', 'nih_chronic_kidney_disease': 'Int64', 'nih_chronic_lung': 'Int64',
    'nih_copd': 'Int64', 'nih_cardiovascular_disease': 'Int64', 'nih_depression': 'Int64',
    'nih_diabetes': 'Int64', 'nih_hypertension': 'Int64', 'nih_immunocompromised': 'Int64',
    'nih_iv_drug_use': 'Int64', 'nih_other_chronic_cond': 'Int64', 'nih_mental_health_disorder': 'Int64',
    'nih_sickle_cell_disease': 'Int64', 'cov_pan_chal_hlth_2': 'Int64', 'cov_pan_chal_med_2': 'Int64',
    'cov_pan_chlng_trans_2': 'Int64', 'cov_tst_mthd_2': 'Int64', 'nih_abdom_pain': 'Int64',
    'nih_cough': 'Int64', 'nih_diff_breath': 'Int64', 'nih_fatigue': 'Int64', 'nih_fever_chills': 'Int64',
    'covid_had': 'Int64', 'nih_headache': 'Int64', 'nih_muscle_ache': 'Int64',
    'nih_nausea_vomiting_diarrhea': 'Int64', 'nih_olfactory': 'Int64', 'nih_other_symp': 'Int64',
    'covid_pandemic_challenges_abod_2': 'Int64', 'covid_pandemic_challenges_food_2': 'Int64',
    'covid_pandemic_challenges_wate_2': 'Int64', 'nih_throat_congestion_nose': 'Int64',
    'nih_skin_rash': 'Int64', 'covid_test_approval': 'Int64', 'covid_test_collection_setting': 'Int64',
    'covid_test_performed_location': 'Int64', 'covid_test_result': 'Int64',
    'covid_test_specimen_collector': 'Int64', 'covid_test_specimen_type': 'Int64',
    'covid_test_study_setting': 'Int64', 'covid_test_tar_dis_stat_2___1': 'Int64',
    'covid_test_tar_dis_stat_2___6': 'Int64', 'covid_test_type': 'Int64', 'covid_vaccine': 'Int64',
    'covid_vaccine_course': 'Int64', 'nih_employment': 'Int64', 'data_source': 'Int64', 'nih_education': 'Int64',
    'employed_ew': 'Int64', 'employed_healthcare_2': 'Int64', 'family_income': 'Int64',
    'fi_afford_balanced_meals': 'Int64', 'fi_change_diet': 'Int64', 'fi_change_diet_frequency': 'Int64',
    'fi_eat_less': 'Int64', 'fi_food_money_frequency': 'Int64', 'fi_high_quality_2': 'Int64',
    'fi_hungry': 'Int64', 'flu_vaccine_season_3': 'Int64', 'flu_vaccinehistind_2': 'Int64',
    'nih_insurance': 'Int64', 'hi_loss_covid': 'Int64', 'household_famgen_3': 'Int64',
    'household_homeless': 'Int64', 'housing_concerns': 'Int64', 'isolate_maintain_job': 'Int64',
    'jobloss_covid19_2': 'Int64', 'language_english': 'Int64', 'language_home___1': 'Int64',
    'language_home___5': 'Int64', 'language_home___6': 'Int64', 'language_home___7': 'Int64',
    'language_home___9': 'Int64', 'language_home___90': 'Int64', 'language_home___99': 'Int64',
    'nih_lifetime_use_alcohol': 'Int64', 'missed_procedure_2': 'Int64', 'positivemonth_covidtest_2': 'Int64',
    'positiveyear_covidtest_3': 'Int64', 'nih_pregnancy': 'Int64', 'quarantine_maintain_job': 'Int64',
    'race_ethn_asian_detail_3___1': 'Int64', 'race_ethn_asian_detail_3___2': 'Int64',
    'race_ethn_asian_detail_3___3': 'Int64', 'race_ethn_asian_detail_3___4': 'Int64',
    'race_ethn_asian_detail_3___5': 'Int64', 'nih_ethnicity': 'Int64',
    'race_ethn_hispanic_detail_2___1': 'Int64', 'race_ethn_hispanic_detail_2___2': 'Int64',
    'race_ethn_hispanic_detail_2___3': 'Int64', 'race_ethn_hispanic_detail_2___4': 'Int64',
    'race_ethn_islander_detail_2___1': 'Int64', 'race_ethn_islander_detail_2___10': 'Int64',
    'race_ethn_islander_detail_2___11': 'Int64', 'race_ethn_islander_detail_2___12': 'Int64',
    'race_ethn_islander_detail_2___13': 'Int64', 'race_ethn_islander_detail_2___2': 'Int64',
    'race_ethn_islander_detail_2___3': 'Int64', 'race_ethn_islander_detail_2___4': 'Int64',
    'race_ethn_islander_detail_2___5': 'Int64', 'race_ethn_islander_detail_2___6': 'Int64',
    'race_ethn_islander_detail_2___7': 'Int64', 'race_ethn_islander_detail_2___8': 'Int64',
    'race_ethn_islander_detail_2___9': 'Int64', 'recentmonth_covidtest_2': 'Int64',
    'recentresult_covidtest': 'Int64', 'recentyear_covidtest_3': 'Int64', 'nih_disability': 'Int64',
    'nih_health_status': 'Int64', 'smoke_years': 'float64', 'nih_history_smoking': 'Int64',
    'smoker_number': 'float64', 'stable_housing': 'Int64', 'take_presc_meds': 'Int64',
    'test_accesseasy_2': 'Int64', 'test_accesswhere_2': 'Int64', 'test_intent_2': 'Int64',
    'test_pbenearound_2': 'Int64', 'test_pbeneexposed_2': 'Int64', 'test_pbenepos_2': 'Int64',
    'test_pbenesafe_2': 'Int64', 'test_pbenework_2': 'Int64', 'test_pbeneworry_2': 'Int64',
    'test_percvaccuneg_2': 'Int64', 'test_percvaccupos_2': 'Int64', 'test_priskcontact_2': 'Int64',
    'test_priskknow_2': 'Int64', 'test_prisklater_2': 'Int64', 'test_prisknohelp_2': 'Int64',
    'test_prisknosymp_2': 'Int64', 'test_priskouch_2': 'Int64', 'test_resneg_2___1': 'Int64',
    'test_resneg_2___2': 'Int64', 'test_resneg_2___3': 'Int64', 'test_resneg_2___4': 'Int64',
    'test_respos_2___1': 'Int64', 'test_respos_2___2': 'Int64', 'test_respos_2___3': 'Int64',
    'tested_for_covid': 'Int64', 'tested_positive_for_covid': 'Int64', 'trust_coll_2': 'Int64',
    'trust_cortf_2': 'Int64', 'trust_doc_2': 'Int64', 'trust_fam_2': 'Int64', 'trust_flead_2': 'Int64',
    'trust_news_2': 'Int64', 'trust_social_2': 'Int64', 'trust_usgov_2': 'Int64', 'vaccine_avail': 'Int64',
    'vaccine_concerns_3___1': 'Int64', 'vaccine_concerns_3___10': 'Int64', 'vaccine_concerns_3___2': 'Int64',
    'vaccine_concerns_3___3': 'Int64', 'vaccine_concerns_3___4': 'Int64', 'vaccine_concerns_3___5': 'Int64',
    'vaccine_concerns_3___6': 'Int64', 'vaccine_concerns_3___7': 'Int64', 'vaccine_concerns_3___8': 'Int64',
    'vaccine_concerns_3___9': 'Int64', 'vaccine_reasons_3___1': 'Int64', 'vaccine_reasons_3___2': 'Int64',
    'vaccine_reasons_3___3': 'Int64', 'vaccine_reasons_3___4': 'Int64', 'vaccine_reasons_3___5': 'Int64',
    'vaccine_reasons_3___6': 'Int64', 'vaccine_reasons_3___7': 'Int64', 'vaccine_reasons_3___8': 'Int64',
    'vaccine_reasons_3___9': 'Int64', 'nih_vape_freq': 'Int64', 'vaper_years': 'float64',
    'work_closecont_2': 'Int64', 'work_ppe_2': 'Int64', 'work_wash_2': 'Int64', 'nih_zip': 'string',
    'nih_weight': 'Int64', 'nih_height': 'Int64', 'nih_race': 'Int64', 'nih_alcohol_yn': 'Int64',
    'nih_cig_smoke_freq': 'Int64', 'nih_smoking_yn': 'Int64'
    }
print(f'total number of columns being mapped: {len(col_type_mapping)}')

data_path = Path("/home", "sagemaker-user", "phs002526", "project26_DATA_transformcopy_v3_processed.csv")
df = pd.read_csv(data_path, dtype=col_type_mapping)
print(f'initial input data shape: {df.shape}')
df = df.reset_index(drop=True)
print(f'input data shape after reset_index: {df.shape}')
print(df.columns)

# for model comparison for covid_test_result prediction, nulls from target feature should be removed and all-null columns should be removed as well
target_feature = 'tested_positive_for_covid'
df = df.dropna(subset=[target_feature])
print('input data shape after dropna target feature: {df.shape}')
df = df[(df[target_feature] != 98) & (df[target_feature] != 99)]
print(f'after removing null and extraneous rows, df.shape: {df.shape}')
null_only_cols = df.columns[df.isna().all()].tolist()
print(f'null-only columns: {null_only_cols}')
df = df.drop(columns=null_only_cols)
df = df.drop(columns='index')
df = df.reset_index(drop=True)
print(f'input dataframe shape after processing: {df.shape}')
print(df[target_feature].value_counts())
df.to_csv(f'{os.path.splitext(data_path)[0]}_{target_feature}.csv', index=False)
pd.set_option('display.max_columns', None)
df.describe()

total number of columns being mapped: 187
initial input data shape: (23111, 188)
input data shape after reset_index: (23111, 188)
Index(['index', 'nih_age', 'nih_alcohol_frequency', 'alcohol_drinksperday',
       'nih_sex', 'nih_asthma', 'nih_alc_sub_abuse', 'nih_autoimm',
       'nih_cancer_past_yr', 'nih_chronic_kidney_disease',
       ...
       'work_closecont_2', 'work_ppe_2', 'work_wash_2', 'nih_zip',
       'nih_weight', 'nih_height', 'nih_race', 'nih_alcohol_yn',
       'nih_cig_smoke_freq', 'nih_smoking_yn'],
      dtype='object', length=188)
input data shape after dropna target feature: {df.shape}
after removing null and extraneous rows, df.shape: (3651, 188)
null-only columns: ['covid_test_approval', 'covid_test_collection_setting', 'covid_test_performed_location', 'covid_test_result', 'covid_test_specimen_collector', 'covid_test_specimen_type', 'covid_test_study_setting', 'covid_test_tar_dis_stat_2___1', 'covid_test_tar_dis_stat_2___6', 'covid_test_type', 'race_ethn_islande

,nih_age,nih_alcohol_frequency,alcohol_drinksperday,nih_sex,nih_asthma,nih_alc_sub_abuse,nih_autoimm,nih_cancer_past_yr,nih_chronic_kidney_disease,nih_chronic_lung,nih_copd,nih_cardiovascular_disease,nih_depression,nih_diabetes,nih_hypertension,nih_immunocompromised,nih_iv_drug_use,nih_other_chronic_cond,nih_mental_health_disorder,nih_sickle_cell_disease,cov_pan_chal_hlth_2,cov_pan_chal_med_2,cov_pan_chlng_trans_2,cov_tst_mthd_2,nih_abdom_pain,nih_cough,nih_diff_breath,nih_fatigue,nih_fever_chills,covid_had,nih_headache,nih_muscle_ache,nih_nausea_vomiting_diarrhea,nih_olfactory,nih_other_symp,covid_pandemic_challenges_abod_2,covid_pandemic_challenges_food_2,covid_pandemic_challenges_wate_2,nih_throat_congestion_nose,nih_skin_rash,covid_vaccine,covid_vaccine_course,nih_employment,data_source,nih_education,employed_ew,employed_healthcare_2,family_income,fi_afford_balanced_meals,fi_change_diet,fi_change_diet_frequency,fi_eat_less,fi_food_money_frequency,fi_high_quality_2,fi_hungry,flu_vaccine_season_3,flu_vaccinehistind_2,nih_insurance,hi_loss_covid,household_famgen_3,household_homeless,housing_concerns,isolate_maintain_job,jobloss_covid19_2,language_english,language_home___1,language_home___5,language_home___6,language_home___7,language_home___9,language_home___90,language_home___99,nih_lifetime_use_alcohol,missed_procedure_2,positivemonth_covidtest_2,positiveyear_covidtest_3,nih_pregnancy,quarantine_maintain_job,race_ethn_asian_detail_3___1,race_ethn_asian_detail_3___2,race_ethn_asian_detail_3___3,race_ethn_asian_detail_3___4,race_ethn_asian_detail_3___5,nih_ethnicity,race_ethn_hispanic_detail_2___1,race_ethn_hispanic_detail_2___2,race_ethn_hispanic_detail_2___3,race_ethn_hispanic_detail_2___4,race_ethn_islander_detail_2___1,race_ethn_islander_detail_2___10,race_ethn_islander_detail_2___11,race_ethn_islander_detail_2___12,race_ethn_islander_detail_2___2,race_ethn_islander_detail_2___3,race_ethn_islander_detail_2___4,race_ethn_islander_detail_2___5,race_ethn_islander_detail_2___6,recentmonth_covidtest_2,recentresult_covidtest,recentyear_covidtest_3,nih_disability,nih_health_status,smoke_years,nih_history_smoking,smoker_number,stable_housing,take_presc_meds,test_accesseasy_2,test_accesswhere_2,test_intent_2,test_pbenearound_2,test_pbeneexposed_2,test_pbenepos_2,test_pbenesafe_2,test_pbenework_2,test_pbeneworry_2,test_percvaccuneg_2,test_percvaccupos_2,test_priskcontact_2,test_priskknow_2,test_prisklater_2,test_prisknohelp_2,test_prisknosymp_2,test_priskouch_2,test_resneg_2___1,test_resneg_2___2,test_resneg_2___3,test_resneg_2___4,test_respos_2___1,test_respos_2___2,test_respos_2___3,tested_for_covid,tested_positive_for_covid,trust_coll_2,trust_cortf_2,trust_doc_2,trust_fam_2,trust_flead_2,trust_news_2,trust_social_2,trust_usgov_2,vaccine_avail,vaccine_concerns_3___1,vaccine_concerns_3___10,vaccine_concerns_3___2,vaccine_concerns_3___3,vaccine_concerns_3___4,vaccine_concerns_3___5,vaccine_concerns_3___6,vaccine_concerns_3___7,vaccine_concerns_3___8,vaccine_concerns_3___9,vaccine_reasons_3___1,vaccine_reasons_3___2,vaccine_reasons_3___3,vaccine_reasons_3___4,vaccine_reasons_3___5,vaccine_reasons_3___6,vaccine_reasons_3___7,vaccine_reasons_3___8,vaccine_reasons_3___9,nih_vape_freq,vaper_years,work_closecont_2,work_ppe_2,work_wash_2,nih_weight,nih_height,nih_race,nih_alcohol_yn,nih_cig_smoke_freq,nih_smoking_yn
count,3651.0,2732.0,1490.0,2675.0,2861.0,2852.0,2844.0,2858.0,2652.0,2652.0,2861.0,2861.0,2904.0,2844.0,2844.0,2861.0,2844.0,2862.0,2876.0,2844.0,2240.0,2240.0,2237.0,2667.0,115.0,228.0,74.0,229.0,96.0,3014.0,220.0,103.0,80.0,12.0,1048.0,2240.0,2240.0,2239.0,79.0,37.0,3568.0,367.0,3367.0,637.0,2297.0,1573.0,387.0,3389.0,3355.0,3328.0,846.0,772.0,3330.0,3244.0,3325.0,1243.0,1691.0,3341.0,3288.0,1832.0,24.0,3276.0,1576.0,2403.0,873.0,8.0,5.0,12.0,5.0,2517.0,34.0,2517.0,2732.0,3273.0,418.0,25.0,1071.0,3473.0,457.0,579.0,531.0,80.0,2.0,39.0,4.0,4.0,4.0,4.0,1346.0,2.0,1.0,2.0,171.0,87.0,10.0,3.0,1.0,3155.0,3430.0,136.0,3479.0,3475.

In [3]:
continuous_feats: list[str] = [
    'nih_height', 'nih_age', 'vaper_years', 'smoke_years', 'nih_weight', 'smoker_number'
]

# These features have been decided to be personally-identifying and, if they
# have not already been anonymized, will be replaced with novel nominals.
sensitive_nominal_feats: dict[str, str] = {
    "nih_zip": "int-id"
}

# These are non-personally-identifying nominals. Their values may be present in
# the resulting synthetic data, but necessary to maintain the predictiveness of
# the resulting synthetic data.
nominal_feats = list(set(df.columns) - set(continuous_feats) - set(sensitive_nominal_feats.keys()))
print(nominal_feats)

['positivemonth_covidtest_2', 'nih_fatigue', 'test_percvaccupos_2', 'trust_flead_2', 'language_home___99', 'nih_race', 'nih_alc_sub_abuse', 'nih_fever_chills', 'language_home___5', 'covid_had', 'race_ethn_islander_detail_2___11', 'nih_skin_rash', 'cov_pan_chal_med_2', 'covid_pandemic_challenges_abod_2', 'nih_nausea_vomiting_diarrhea', 'nih_diff_breath', 'take_presc_meds', 'vaccine_reasons_3___6', 'test_percvaccuneg_2', 'vaccine_reasons_3___5', 'fi_high_quality_2', 'nih_headache', 'employed_ew', 'flu_vaccine_season_3', 'trust_usgov_2', 'race_ethn_asian_detail_3___5', 'nih_depression', 'test_priskknow_2', 'nih_education', 'test_pbenework_2', 'nih_chronic_lung', 'vaccine_concerns_3___8', 'race_ethn_hispanic_detail_2___1', 'jobloss_covid19_2', 'missed_procedure_2', 'work_ppe_2', 'language_home___1', 'race_ethn_islander_detail_2___5', 'nih_other_symp', 'employed_healthcare_2', 'tested_for_covid', 'recentyear_covidtest_3', 'vaccine_concerns_3___5', 'fi_afford_balanced_meals', 'housing_concer

In [4]:
# Compute feature attributes
features = infer_feature_attributes(
    df,
    types={
        "continuous": continuous_feats,
        "nominal": list(sensitive_nominal_feats.keys()) + nominal_feats
    },
    # Let's use tight-bounds for the continuous
    tight_bounds=continuous_feats,
)

# Non-sensitive nominals should be marked as such
for nominal in nominal_feats:
    features[nominal]["non_sensitive"] = True

if ALREADY_ANONYMIZED:
    # If data is already anonymized, mark the "sensitive" features non-sensitive too.
    for nominal in sensitive_nominal_feats.keys():
        features[nominal]["non_sensitive"] = True
else:
    # Let's mark the sensitive nominals with the desired subtypes
    for nominal, subtype in sensitive_nominal_feats.items():
        features[nominal]["subtype"] = subtype

features.to_dataframe()


type decimal_places bounds                   \
                                                    min   max allow_null   
nih_age                continuous            0.0    7.0  89.0       True   
nih_alcohol_frequency     nominal            0.0    NaN   NaN       True   
alcohol_drinksperday      nominal            0.0    NaN   NaN       True   
nih_sex                   nominal            0.0    NaN   NaN       True   
nih_asthma                nominal            0.0    NaN   NaN       True   
...                           ...            ...    ...   ...        ...   
nih_height             continuous            0.0   38.0  81.0       True   
nih_race                  nominal            0.0    NaN   NaN       True   
nih_alcohol_yn            nominal            0.0    NaN   NaN       True   
nih_cig_smoke_freq        nominal            0.0    NaN   NaN       True   
nih_smoking_yn            nominal            0.0    NaN   NaN       True   

                                                data_type non_sensitive  \
                      observed_min observed_max                           
nih_age                        7.0         89.0    number           NaN   
nih_alcohol_frequency          NaN          NaN    number          True   
alcohol_drinksperday           NaN          NaN    number          True   
nih_sex                        NaN          NaN    number          True   
nih_asthma                     NaN          NaN    number          True   
...                            ...          ...       ...           ...   
nih_height                    38.0         81.0    number           NaN   
nih_race                       NaN          NaN    number          True   
nih_alcohol_yn                 NaN          NaN    number          True   
nih_cig_smoke_freq             NaN          NaN    number          True   
nih_smoking_yn                 NaN          NaN    number          True   

                      original_type       
                          data_type size  
nih_age                     integer  8.0  
nih_alcohol_frequency       integer  8.0  
alcohol_drinksperday        integer  8.0  
nih_sex                     integer  8.0  
nih_asthma                  integer  8.0  
...                             ...  ...  
nih_height                  integer  8.0  
nih_race                    integer  8.0  
nih_alcohol_yn              integer  8.0  
nih_cig_smoke_freq          integer  8.0  
nih_smoking_yn              integer  8.0  

[173 rows x 11 columns]

In [5]:
if not ALREADY_ANONYMIZED:
    # Before continuing, let's substitute sensitive features with replacement nominals.

    # Instantiate the NominalSubstitutionEngine
    substitution_engine = NominalSubstitutionEngine.factory(nominal_substitution_config={
        "nominals": {"default": {"locales": ["en_US"], "substitution": "naive"}},
    })
    df = substitution_engine.apply_substitutions(features, df)

    # Now that these replacements have been made, we no longer need these marked as sensitive.
    for feature in sensitive_nominal_feats.keys():
        features[feature]["subtype"] = None
        features[feature]["non_sensitive"] = True

    # Display the new dataframe and feature attributes
    display(df)
    display(features.to_dataframe())


In [6]:
desired_conviction_list = [0.1, 0.5, 1.0, 3.0, 5.0, 10.0, 25.0]
synthed_dfs: list[pd.DataFrame] = []
validator_results: list[ValidatorResult] = []
metrics = [
    # This provides a privacy metric
    "AnonymityPreservation",
    # These are the 3 most typical used for Accuracy metrics
    "DescriptiveStatistics",
    "JointProbability",
    "ModelComparison",   # This will give us more detailed, per-feature accuracy metrics.
]

# Move desired_conviction loop out to the top so that we don't re-train the
# same trainee over and over.
for desired_conviction in desired_conviction_list:
    display(Markdown(f"## Processing desired_conviction={desired_conviction}"))

    with Synthesizer(privacy_override=True, use_id_privacy=False) as s:
        s.train(df, features=features)

        synth_df = s.synthesize_cases(
            n_samples=df.shape[0],
            desired_conviction=desired_conviction,
            # generate_new_cases="always"
            generate_new_cases="no"
        )
        synthed_dfs.append(synth_df)
        synth_df.to_csv(f'data/phs002526/{target_feature}/gen_new_case_no/synthesized_{desired_conviction}.csv', 
                        index=False)
        with Validator(s, synth_df) as v:
            results = v.run_metrics(metrics)
        validator_results.append(results)

## Processing desired_conviction=0.1

Version 50.3.1 of Howso Engine™ is available. You are using version 50.2.1.


Validator Enterprise is enabled. Running without parallelism.
AnonymityPreservation       : Beginning run
AnonymityPreservation       : Run completed with desirability=5.0
DescriptiveStatistics       : Beginning run
DescriptiveStatistics       : Run completed with desirability=3.473
JointProbability            : Beginning run
JointProbability            : Run completed with desirability=3.734
ModelComparison             : Beginning run
ModelComparison             : Run completed with desirability=0.135
Run finished with overall_desirability=1.72


## Processing desired_conviction=0.5

Validator Enterprise is enabled. Running without parallelism.
AnonymityPreservation       : Beginning run
AnonymityPreservation       : Run completed with desirability=5.0
DescriptiveStatistics       : Beginning run
DescriptiveStatistics       : Run completed with desirability=3.392
JointProbability            : Beginning run
JointProbability            : Run completed with desirability=4.084
ModelComparison             : Beginning run
ModelComparison             : Run completed with desirability=1.165
Run finished with overall_desirability=2.997


...

Validator Enterprise is enabled. Running without parallelism.
AnonymityPreservation       : Beginning run
AnonymityPreservation       : Run completed with desirability=5.0
DescriptiveStatistics       : Beginning run
DescriptiveStatistics       : Run completed with desirability=4.332
JointProbability            : Beginning run
JointProbability            : Run completed with desirability=4.57
ModelComparison             : Beginning run
ModelComparison             : Run completed with desirability=4.583
Run finished with overall_desirability=4.615


## Processing desired_conviction=10.0

Validator Enterprise is enabled. Running without parallelism.
AnonymityPreservation       : Beginning run
AnonymityPreservation       : Run completed with desirability=5.0
DescriptiveStatistics       : Beginning run
DescriptiveStatistics       : Run completed with desirability=4.258
JointProbability            : Beginning run
JointProbability            : Run completed with desirability=4.699
ModelComparison             : Beginning run
ModelComparison             : Run completed with desirability=4.552
Run finished with overall_desirability=4.62


## Processing desired_conviction=25.0

Validator Enterprise is enabled. Running without parallelism.
AnonymityPreservation       : Beginning run
AnonymityPreservation       : Run completed with desirability=5.0
DescriptiveStatistics       : Beginning run
DescriptiveStatistics       : Run completed with desirability=4.419
JointProbability            : Beginning run
JointProbability            : Run completed with desirability=4.793
ModelComparison             : Beginning run
ModelComparison             : Run completed with desirability=4.603
Run finished with overall_desirability=4.699


In [ ]:
# Display the full metric details for each desirability_conviction level
for desired_conviction, synth_df, results in zip(desired_conviction_list, synthed_dfs, validator_results, strict=True):
    display(Markdown(f"## {desired_conviction=}"))
    for metric_name, result in results.items():
        display(metric_name)
        display(result.get_dataframe())

In [ ]:
# Create a summary DataFrame
final_result = pd.DataFrame()
data: dict[str, list] = defaultdict(list)

for desired_conviction, validator_result in zip(desired_conviction_list, validator_results):
    data["desired_conviction"].append(desired_conviction)
    for metric in metrics:
        data[metric].append(validator_result[metric].desirability)

final_result = pd.DataFrame(data)

display(final_result)


In [ ]:
import matplotlib.pyplot as plt

# Melt the dataframe to long format
melted_df = final_result.melt(
    id_vars="desired_conviction",
    value_vars=metrics,
    var_name="Metric",
    value_name="Desirability"
)

# Create grouped bar chart
plt.figure(figsize=(15, 6))
sns.barplot(data=melted_df, x="desired_conviction", y="Desirability", hue="Metric")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(15, 10))
categorical_features = [f for f, a in features.items() if a["type"] in ("nominal", "ordinal")]

gen_df = synthed_dfs[-1]  # Using the one with the highest accuracy (theoretically)

for c, ax in zip(df.columns.drop(categorical_features), axes.flat):
    print(c, df[c].dtypes, df[c].min(), df[c].max())
    if df[c].dtypes != 'string':
        sns.kdeplot(ax=ax, x=df[c], color="dodgerblue", label="full_data", fill=True)
        sns.kdeplot(ax=ax, x=gen_df[c], color="orange", label="full_data", fill=True)
        ax.set_xlabel(c)

fig.legend(labels=["full_data", "synth_data"])
fig.tight_layout()
plt.show()

In [ ]:
# Display the full ModelComparison results dataframe for the final run
with pd.option_context('display.max_rows', None):
    display(validator_results[-1]["ModelComparison"].get_dataframe())